# Анализ сервиса вопросов и ответов по программированию

Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

db_config = {
'user':'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-advanced-sql'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
) 
engine = create_engine(connection_string)

In [2]:
query = '''
SELECT COUNT(id)
FROM stackoverflow.posts
WHERE post_type_id=1 AND (score>300 OR favorites_count>=100 );'''
pd.read_sql_query(query, con = engine) 

,count
0,1355


Cколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [3]:
query = '''
WITH posts AS
(SELECT creation_date::DATE,
        COUNT(id) AS post
FROM stackoverflow.posts        
WHERE post_type_id=1 AND (creation_date::DATE BETWEEN '2008-11-01' AND '2008-11-18')
GROUP BY creation_date::DATE)
SELECT ROUND(AVG(post))
FROM posts;'''
pd.read_sql_query(query, con = engine) 

,round
0,383.0


Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [4]:
query = '''
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.users AS u
LEFT JOIN stackoverflow.badges AS b ON b.user_id = u.id
WHERE b.creation_date::DATE = u.creation_date::DATE;'''
pd.read_sql_query(query, con = engine) 

,count
0,7047


Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [5]:
query = '''
SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.users AS u 
JOIN stackoverflow.posts AS p ON u.id=p.user_id
JOIN stackoverflow.votes AS v ON p.id=v.post_id
WHERE u.display_name = 'Joel Coehoorn';'''
pd.read_sql_query(query, con = engine) 

,count
0,12


Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [6]:
query = '''
SELECT *,
       ROW_NUMBER() OVER (ORDER BY id DESC) AS rank
FROM stackoverflow.vote_types
ORDER BY id;'''
pd.read_sql_query(query, con = engine) 

,id,name,rank
0,1,AcceptedByOriginator,15
1,2,UpMod,14
2,3,DownMod,13
3,4,Offensive,12
4,5,Favorite,11
5,6,Close,10
6,7,Reopen,9
7,8,BountyStart,8
8,9,BountyClose,7
9,10,Deletion,6


Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [7]:
query = '''
WITH vvt AS
(SELECT *
FROM stackoverflow.votes AS v
JOIN stackoverflow.vote_types AS vt ON vt.id=v.vote_type_id
WHERE vt.name = 'Close')
SELECT user_id,
       COUNT(user_id) AS total
FROM vvt 
GROUP BY user_id
ORDER BY total DESC, user_id DESC
LIMIT 10;'''
pd.read_sql_query(query, con = engine) 

,user_id,total
0,20646,36
1,14728,36
2,27163,29
3,41158,24
4,24820,23
5,9345,23
6,3241,23
7,44330,20
8,38426,19
9,19074,19


Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
идентификатор пользователя;
число значков;
место в рейтинге — чем больше значков, тем выше рейтинг.
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [8]:
query = '''
WITH badges AS
(SELECT user_id,
       COUNT(id) AS total
FROM stackoverflow.badges
WHERE creation_date::DATE BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY user_id)
SELECT *,
       DENSE_RANK() OVER (ORDER BY total DESC)
FROM badges
ORDER BY total DESC, user_id
LIMIT 10;'''
pd.read_sql_query(query, con = engine) 

,user_id,total,dense_rank
0,22656,149,1
1,34509,45,2
2,1288,40,3
3,5190,31,4
4,13913,30,5
5,893,28,6
6,10661,28,6
7,33213,25,7
8,12950,23,8
9,25222,20,9


Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
заголовок поста;
идентификатор пользователя;
число очков поста;
среднее число очков пользователя за пост, округлённое до целого числа.
Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [9]:
query = '''
WITH posts AS
(SELECT user_id,
       ROUND(AVG(score)) AS mid
FROM stackoverflow.posts
WHERE score != 0 AND title IS NOT NULL
GROUP BY user_id)
SELECT posts1.title,
       posts1.user_id,
       posts1.score,
       posts.mid
FROM stackoverflow.posts AS posts1
JOIN posts ON posts1.user_id = posts.user_id
WHERE posts1.score != 0 AND posts1.title IS NOT NULL;'''
pd.read_sql_query(query, con = engine) 

,title,user_id,score,mid
0,Diagnosing Deadlocks in SQL Server 2005,1,82,573.0
1,How do I calculate someone's age in C#?,1,1743,573.0
2,Why doesn't IE7 copy <pre><code> blocks to the...,1,37,573.0
3,Calculate relative time in C#,1,1348,573.0
4,Wrapping StopWatch timing with a delegate or l...,1,92,573.0
...,...,...,...,...
47349,Multi-lingual projects in Visual Studio,3742716,1,1.0
47350,Change datatype when importing Excel file into...,3930756,5,5.0
47351,How to host a Silverlight app in a Sharepoint ...,4020932,8,8.0
47352,Getting counts for a paged SQL search stored p...,5696608,2,2.0


Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список

In [10]:
query = '''
WITH badges AS
(SELECT user_id,
       COUNT(user_id) AS total
FROM stackoverflow.badges
GROUP BY user_id
HAVING COUNT(user_id) > 1000)
SELECT posts.title
FROM stackoverflow.posts AS posts
JOIN badges ON badges.user_id=posts.user_id
WHERE title IS NOT NULL;'''
pd.read_sql_query(query, con = engine) 

,title
0,What's the hardest or most misunderstood aspec...
1,What's the strangest corner case you've seen i...
2,Project management to go with GitHub
3,What are the correct version numbers for C#?


Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
пользователям с числом просмотров меньше 100 — группу 3.
Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [11]:
query = '''WITH users AS
(SELECT id,
       views,
       (CASE
          WHEN views >=350 THEN 1
          WHEN views <350 AND views >=100 THEN 2
          WHEN views <100 THEN 3
       END) AS lev
FROM stackoverflow.users
WHERE location LIKE '%%United States%%' AND views !=0),
users2 AS
(SELECT id,
       views,
       lev,
       MAX(views) OVER (PARTITION BY lev) AS mikc
FROM users)
SELECT users2.id,
       users2.views,
       users2.lev
FROM users2
WHERE users2.views=users2.mikc
ORDER BY views DESC, id;'''
pd.read_sql_query(query, con = engine) 

,id,views,lev
0,16587,62813,1
1,9094,349,2
2,9585,349,2
3,15079,349,2
4,33437,349,2
5,3469,99,3
6,4829,99,3
7,19006,99,3
8,22732,99,3
9,403434,99,3


Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [12]:
query = '''
WITH users AS
(SELECT id,
       views,
       (CASE
          WHEN views >=350 THEN 1
          WHEN views <350 AND views >=100 THEN 2
          WHEN views <100 THEN 3
       END) AS lev
FROM stackoverflow.users
WHERE location LIKE '%%United States%%' AND views !=0),
users2 AS
(SELECT id,
       views,
       lev,
       MAX(views) OVER (PARTITION BY lev) AS mikc
FROM users)
SELECT users2.id,
       users2.views,
       users2.lev
FROM users2
WHERE users2.views=users2.mikc
ORDER BY views DESC, id;'''
pd.read_sql_query(query, con = engine) 

,id,views,lev
0,16587,62813,1
1,9094,349,2
2,9585,349,2
3,15079,349,2
4,33437,349,2
5,3469,99,3
6,4829,99,3
7,19006,99,3
8,22732,99,3
9,403434,99,3


Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
номер дня;
число пользователей, зарегистрированных в этот день;
сумму пользователей с накоплением.

In [13]:
query = '''WITH users AS
(SELECT *
FROM stackoverflow.users
WHERE creation_date BETWEEN '2008-11-01' AND '2008-12-01'),
users2 AS
(SELECT EXTRACT(DAY FROM creation_date) AS day,
        COUNT(id) AS summa
FROM users
GROUP BY EXTRACT(DAY FROM creation_date))
SELECT day,
       summa,
       SUM(summa) OVER (ORDER BY day ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
FROM users2
ORDER BY day;'''
pd.read_sql_query(query, con = engine) 

,day,summa,sum
0,1.0,34,34.0
1,2.0,48,82.0
2,3.0,75,157.0
3,4.0,192,349.0
4,5.0,122,471.0
5,6.0,132,603.0
6,7.0,104,707.0
7,8.0,42,749.0
8,9.0,45,794.0
9,10.0,93,887.0


Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
идентификатор пользователя;
разницу во времени между регистрацией и первым постом.

In [14]:
query = '''SELECT DISTINCT posts.user_id,
       MIN(posts.creation_date) OVER (PARTITION BY posts.user_id) - users.creation_date 
FROM stackoverflow.posts AS posts
LEFT JOIN stackoverflow.users AS users ON posts.user_id=users.id
GROUP BY posts.user_id, posts.creation_date, users.creation_date;'''
pd.read_sql_query(query, con = engine) 

,user_id,?column?
0,1,0 days 09:18:29
1,2,0 days 14:37:03
2,3,3 days 16:17:09
3,4,15 days 05:44:22
4,5,1 days 14:57:51
...,...,...
18844,3644960,0 days 00:00:00
18845,3742716,66 days 21:15:48
18846,3930756,8 days 02:11:55
18847,4020932,47 days 18:30:34


Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [15]:
query = '''WITH posts AS
(SELECT *
FROM stackoverflow.posts
WHERE creation_date BETWEEN '2008-01-01' AND '2008-12-31')
SELECT DATE_TRUNC('month', creation_date)::DATE,
       SUM(views_count) 
FROM posts
GROUP BY DATE_TRUNC('month', creation_date)::DATE
ORDER BY SUM(views_count) DESC;'''
pd.read_sql_query(query, con = engine) 

,date_trunc,sum
0,2008-09-01,452928568
1,2008-10-01,365400138
2,2008-11-01,221759651
3,2008-12-01,197792841
4,2008-08-01,131367083
5,2008-07-01,669895


Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [16]:
query = '''SELECT u.display_name,
       COUNT(DISTINCT p.user_id)
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u ON p.user_id=u.id
JOIN stackoverflow.post_types AS pt ON pt.id=p.post_type_id
WHERE p.creation_date::DATE BETWEEN u.creation_date::DATE AND (u.creation_date::DATE + INTERVAL '1 month') 
      AND pt.TYPE LIKE '%%Answer%%'
GROUP BY u.display_name
HAVING COUNT(p.id) > 100
ORDER BY u.display_name;'''
pd.read_sql_query(query, con = engine) 

,display_name,count
0,1800 INFORMATION,1
1,Adam Bellaire,1
2,Adam Davis,1
3,Adam Liss,1
4,Alan,8
...,...,...
74,lomaxx,1
75,mattlant,1
76,paxdiablo,1
77,tvanfosson,1



Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [17]:
query = '''WITH id_norm AS
(SELECT DISTINCT u.id
FROM stackoverflow.users AS u
JOIN stackoverflow.posts AS p ON u.id=p.user_id
WHERE CAST(DATE_TRUNC('month', u.creation_date) AS DATE) = '2008-09-01'
AND CAST(DATE_TRUNC('month', p.creation_date) AS DATE) = '2008-12-01')
SELECT CAST(DATE_TRUNC('month', p.creation_date) AS DATE) AS months,
       COUNT(p.id) AS total_posts
FROM stackoverflow.posts AS p
WHERE CAST(DATE_TRUNC('year', p.creation_date) AS DATE) = '2008-01-01'
      AND p.user_id IN (SELECT * FROM id_norm)
GROUP BY months
ORDER BY months DESC;'''
pd.read_sql_query(query, con = engine) 

,months,total_posts
0,2008-12-01,17641
1,2008-11-01,18294
2,2008-10-01,27171
3,2008-09-01,24870
4,2008-08-01,32


Используя данные о постах, выведите несколько полей:
идентификатор пользователя, который написал пост;
дата создания поста;
количество просмотров у текущего поста;
сумму просмотров постов автора с накоплением.
Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [18]:
query = '''SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts
ORDER BY user_id, creation_date;'''
pd.read_sql_query(query, con = engine) 

,user_id,creation_date,views_count,sum
0,1,2008-07-31 23:41:00,480476,480476
1,1,2008-07-31 23:55:38,136033,616509
2,1,2008-07-31 23:56:41,0,616509
3,1,2008-08-04 02:45:08,0,616509
4,1,2008-08-04 04:31:03,0,616509
...,...,...,...,...
243791,5696608,2008-12-23 16:00:37,0,2804
243792,5696608,2008-12-23 17:35:09,0,2804
243793,5696608,2008-12-24 01:02:48,0,2804
243794,5696608,2008-12-30 14:34:45,0,2804


Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [19]:
query = '''WITH norm AS
(WITH posts AS
(SELECT DISTINCT user_id,
       creation_date::date AS dat
FROM stackoverflow.posts
WHERE creation_date BETWEEN '2008-12-01' AND '2008-12-07'
GROUP BY creation_date, user_id)
SELECT COUNT(dat) OVER (PARTITION BY user_id) AS mid
FROM posts)
SELECT ROUND(AVG(mid))
FROM norm;'''
pd.read_sql_query(query, con = engine) 

,round
0,2.0



На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
номер месяца;
количество постов за месяц;
процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.
Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [20]:
query = '''WITH posts AS
(SELECT EXTRACT(MONTH FROM creation_date) AS month,
       COUNT(id) AS total
FROM stackoverflow.posts
WHERE creation_date BETWEEN '2008-09-01' AND '2008-12-31'
GROUP BY EXTRACT(MONTH FROM creation_date))
SELECT *,
       ROUND(((total::numeric / LAG(total) OVER (ORDER BY month)) - 1) * 100, 2)
FROM posts;'''
pd.read_sql_query(query, con = engine) 

,month,total,round
0,9.0,70371,NaN
1,10.0,63102,-10.33
2,11.0,46975,-25.56
3,12.0,44592,-5.07


Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
номер недели;
дата и время последнего поста, опубликованного на этой неделе.

In [21]:
query = '''SELECT DISTINCT EXTRACT(WEEK FROM creation_date::date),
       MAX(creation_date) OVER (PARTITION BY EXTRACT(WEEK FROM creation_date::date))
FROM stackoverflow.posts 
WHERE DATE_TRUNC('month', creation_date) = '2008-10-01'
    AND user_id IN (SELECT user_id
                                FROM (SELECT user_id,
                                       COUNT(id)
                                FROM stackoverflow.posts
                                GROUP BY 1
                                ORDER BY 2 DESC) AS a
                                LIMIT 1);'''
pd.read_sql_query(query, con = engine) 

,date_part,max
0,40.0,2008-10-05 09:00:58
1,41.0,2008-10-12 21:22:23
2,42.0,2008-10-19 06:49:30
3,43.0,2008-10-26 21:44:36
4,44.0,2008-10-31 22:16:01
